# Linear Mixed Models

Consider a linear mixed effects model
$$
	y_i = \mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i, \quad i=1,\ldots,n,
$$
where $\epsilon_i$ are independent normal errors $N(0,\sigma_0^2)$, $\beta \in \mathbb{R}^p$ are fixed effects, and $\gamma \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \sigma_1^2 \mathbf{I}_q$) independent of $\epsilon_i$. 

0. Show that 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$
where $\mathbf{y} = (y_1, \ldots, y_n)^T \in \mathbb{R}^n$, $\mathbf{X} = (\mathbf{x}_1, \ldots, \mathbf{x}_n)^T \in \mathbb{R}^{n \times p}$, and $\mathbf{Z} = (\mathbf{z}_1, \ldots, \mathbf{z}_n)^T \in \mathbb{R}^{n \times q}$. 

0. Write a function, with interface 
    ```julia
    logpdf_mvn(y::Vector, Z::Matrix, σ0::Number, σ1::Number),
    ```
that evaluates the log-density of a multivariate normal with mean $\mathbf{0}$ and covariance $\sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$ at $\mathbf{y}$. Make your code efficient in the $n \gg q$ case. 

0. Compare your result (both accuracy and timing) to the [Distributions.jl](http://distributionsjl.readthedocs.io/en/latest/multivariate.html#multivariate-normal-distribution) package using following data.  
    ```julia
    using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)

    # check you answer matches that from Distributions.jl
    @show logpdf_mvn(y, Z, σ0, σ1)
    @show logpdf(mvn, y)

    # benchmark
    @benchmark logpdf_mvn(y, Z, σ0, σ1)
    @benchmark logpdf(mvn, y)
    ```

1.Show that 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$
where $\mathbf{y} = (y_1, \ldots, y_n)^T \in \mathbb{R}^n$, $\mathbf{X} = (\mathbf{x}_1, \ldots, \mathbf{x}_n)^T \in \mathbb{R}^{n \times p}$, and $$\mathbf{Z} = (\mathbf{z}_1, \ldots, \mathbf{z}_n)^T \in \mathbb{R}^{n \times q}$$

Solution:

I infer the distribution of $\mathbf{y}$ by infering the mean and variance separately.

For mean of $\mathbf{y}$:
$$E(\mathbf{y}) = E(\mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i)$$
$$= E(\mathbf{x}_i^T \beta) + E(\mathbf{z}_i^T \gamma) + E(\epsilon_i)$$
$$= \mathbf{X}\beta + 0 + \mathbf{0}_q $$
$$= \mathbf{X}\beta $$

For variance of $\mathbf{y}$:
$$var(\mathbf{y}) = var(\mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i)$$
Because $\mathbf{x}_i^T \beta$ is fixed, so $var(\mathbf{x}_i^T \beta)=0$ and $\mathbf{x}_i^T \beta$ is independent of $\mathbf{z}_i^T \gamma$ and $\epsilon_i$. So the covariance is also 0.

Therefore, we can simplify $var(\mathbf{y})$ to 
$$var(\mathbf{y}) = var(\mathbf{z}_i^T \gamma) + var(\epsilon_i) + 2cov(\mathbf{z}_i^T \gamma, \epsilon_i)$$
Because $\mathbf{z}_i^T \gamma$ and $\epsilon_i$ are independent, so $cov(\mathbf{z}_i^T \gamma, \epsilon_i)=0$.
$$var(\mathbf{y}) = var(\mathbf{z}_i^T \gamma) + var(\epsilon_i)$$
$$= \sigma_1^2 \mathbf{Z} \mathbf{Z}^T + \sigma_0^2 \mathbf{I}_n$$

So 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right)
$$


2.Write a function, with interface that evaluates the log-density of a multivariate normal with mean $\mathbf{0}$ and covariance $\sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$ at $\mathbf{y}$. Make your code efficient in the $n \gg q$ case. 
    
multivariant normal the PDF is 
$$det(2\pi\Sigma)^{-1/2}e^{-1/2(y-\mu)'\Sigma^{-1}(y-\mu)}$$
where $\mu = 0$, and $\Sigma = \sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$

I first simplify different components separately:
$$det(2\pi\Sigma)=det(2\pi\sigma_0^2I+2\pi\sigma_{1}^2ZZ^T)$$
$$det(2\pi\Sigma)=det(2\pi\sigma_0^2I+2\pi\sigma_{1}^2ZZ^T)$$
$$=2\pi\sigma_0^2 det(I)+2\pi\sigma_{1}^2det(ZZ^T)$$
$$=2\pi\sigma_0^2 +2\pi\sigma_{1}^2det(ZZ^T)$$

Let's move to another part:
$$-\frac{1}{2}y'(\sigma_0^2I+\sigma_1^2ZZ^T)^{-1}y$$

This part I notice the $\sigma_0^2I+\sigma_1^2ZZ^T$ is positive definite square matrix, so we can use cholesky decomposition to decompose it first and use the upper and lower triagle matrix to times the $y'$ and $y$ later.

In [9]:
# this is an efficiency-savvy person 

function logpdf_mvn(y::Vector, Z::Matrix, σ0::Number, σ1::Number)
    
    s0 = σ0^2
    s1 = σ1^2
    
    #Σ = s0 * I + s1 * At_mul_B(Z,Z)
    #Σ = s0*I + s1*Z*Z.'

    
    #n = length(y)
    #Σchol = cholfact(Symmetric(Σ))
    
    #logdet(Σchol) = s0 * logdet(diag(n) + s0 * Z' * Z 
    n = size(Z,1)
    q = size(Z,2)
    
    logdetchol = log(s0)*n + logdet(eye(q) + s1/s0 * At_mul_B(Z,Z) )
    
    
    chol=cholfact(eye(q)+1/s0*At_mul_B(Z,Z))
    #inversechol = 1/s0 - 1/s0*(Z*chol[:U]) * (chol[:L]*Z').*s1/s0
    middle = eye(q)+s1/s0*Z'*Z
    cholmiddle = cholfact(Symmetric(eye(q)+s1/s0*Z'*Z))
    inversechol = 1/s0*eye(n) - s1/(s0^2)*(Z*inv(cholmiddle)*Z') #I know here I should use cholesky to replace inv,
                                                                #but there is some error for the final result if I use
                                                                #that...feel I should improve my understanding of using 
                                                                #chelesky to inv
    #inversechol = 1/s0*eye(n) - s1/(s0^2)*sum(abs2,cholmiddle[:L]\cholmiddle)
    #inversechol = 1/s0*eye(n) - s1/(s0^2)*(Z*inv(eye(q)+s1/s0*Z'*Z)*Z')
    last = y' * inversechol * y
    
    - (n//2) * log(2π) - (1//2) * logdetchol - (1//2)*last
    
    #- (n//2) * log(2π) - (1//2) * logdet(Σchol) - (1//2) * dot(y, Σchol \ y)
   
    
end


logpdf_mvn (generic function with 2 methods)

3.Compare your result (both accuracy and timing) to the [Distributions.jl]

In [10]:
using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)
    @show logpdf_mvn(y, Z, σ0, σ1)
    @benchmark logpdf_mvn(y, Z, σ0, σ1)

logpdf_mvn(y, Z, σ0, σ1) = -1571.5736734654702


BenchmarkTools.Trial: 
  memory estimate:  153.54 MiB
  allocs estimate:  58
  --------------
  minimum time:     65.797 ms (28.49% GC)
  median time:      141.278 ms (66.49% GC)
  mean time:        147.715 ms (66.90% GC)
  maximum time:     189.022 ms (70.83% GC)
  --------------
  samples:          34
  evals/sample:     1

In [3]:
using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)   
    @show logpdf(mvn, y)    
    @benchmark logpdf(mvn, y)

logpdf(mvn, y) = -1571.5736734654186


BenchmarkTools.Trial: 
  memory estimate:  15.78 KiB
  allocs estimate:  3
  --------------
  minimum time:     5.620 ms (0.00% GC)
  median time:      5.826 ms (0.00% GC)
  mean time:        5.955 ms (0.00% GC)
  maximum time:     9.236 ms (0.00% GC)
  --------------
  samples:          837
  evals/sample:     1